## 1. Data Upload

In [1]:
import pandas as pd
import numpy as np
from io import StringIO
from sklearn.manifold import MDS
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize

path_csv = '.'

# Read the CSV files into pandas dataframes
input_formal_model = pd.read_csv(f"{path_csv}/formal_model.csv")

In [2]:
input_formal_model

,ID,Target,f1,f2,f3,f4,f5
0,1,1,2.8,-1.8,-2.800000,1.300000,0.4
1,2,1,2.9,-1.9,-2.900000,1.400000,0.5
2,3,1,3.0,-2.0,-3.000000,1.500000,0.6
3,4,1,3.1,-2.1,-3.100000,1.600000,0.7
4,5,1,3.2,-2.2,-3.200000,1.700000,0.8
5,6,2,-1.6,-2.5,1.500000,0.200000,0.6
6,7,2,-1.3,-2.7,1.300000,0.400000,0.8
7,8,2,-1.0,-3.0,1.500000,0.600000,1.0
8,9,2,-0.7,-3.2,1.700000,0.800000,1.2
9,10,2,-0.5,-3.5,1.900000,1.000000,1.4


## 2. Multi-dimensional scaling

In [3]:
# Extract the necessary subtables
formal_model_features = input_formal_model.drop('Target', axis=1).drop('ID', axis=1)
formal_model_with_id = input_formal_model.drop('Target', axis=1)
formal_model_id = input_formal_model['ID']
formal_model_target = input_formal_model['Target']

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(formal_model_features)

# Apply MDS to reduce the dimensions to 2
mds = MDS(n_components=2, normalized_stress='auto')
X_mds = mds.fit_transform(X_scaled)

# Convert the reduced data back to a pandas DataFrame
formal_model_reduced = pd.DataFrame(X_mds, columns=["x1", "x2"])


In [4]:
formal_model_with_id.to_csv('formal_model_with_id.csv', index=False)
formal_model_with_id

,ID,f1,f2,f3,f4,f5
0,1,2.8,-1.8,-2.800000,1.300000,0.4
1,2,2.9,-1.9,-2.900000,1.400000,0.5
2,3,3.0,-2.0,-3.000000,1.500000,0.6
3,4,3.1,-2.1,-3.100000,1.600000,0.7
4,5,3.2,-2.2,-3.200000,1.700000,0.8
5,6,-1.6,-2.5,1.500000,0.200000,0.6
6,7,-1.3,-2.7,1.300000,0.400000,0.8
7,8,-1.0,-3.0,1.500000,0.600000,1.0
8,9,-0.7,-3.2,1.700000,0.800000,1.2
9,10,-0.5,-3.5,1.900000,1.000000,1.4


## 3. Visual analytics

In [5]:
def interpolate_coordinates(x_1, x_n, n):
    """
    This function calculates the values of the interpolate_coordinates
    of a line based on its first and last coordinates.
    :param x_1:
    :param x_n:
    :param n:
    :return:
    """
    # Ensure n is greater than 1 to avoid division by zero
    if n <= 1:
        raise ValueError("n must be greater than 1 to calculate intermediate points.")

    # List to hold all the coordinates including the first and the last
    inter_coordinates = []

    # Calculate each intermediate coordinate
    for i in range(0, n):
        x_i = x_1 + ((x_n - x_1) * i) / (n - 1)
        inter_coordinates.append(x_i)

    return inter_coordinates


In [6]:
x_1_first = 2
x_1_last = -2
x_2_first = -2
x_2_last = 0
num_points = 5

x_1_inter_coordinates = interpolate_coordinates(x_1_first, x_1_last, num_points)
x_2_inter_coordinates = interpolate_coordinates(x_2_first, x_2_last, num_points)
# print(x_1_inter_coordinates)
# print(x_2_inter_coordinates)

# Create a DataFrame from the coordinates
coordinates_df = pd.DataFrame({'x1': x_1_inter_coordinates, 'x2': x_2_inter_coordinates})
coordinates_df


,x1,x2
0,2.0,-2.0
1,1.0,-1.5
2,0.0,-1.0
3,-1.0,-0.5
4,-2.0,0.0


In [7]:
import matplotlib.pyplot as plt

# Include class colors and a legend
colors = {1: 'red', 2: 'green', 3: 'blue'}  # Assigning colors to each class

plt.figure(figsize=(10, 6))

# Iterate over each class and plot each class with a different color
for cls in input_formal_model['Target'].unique():
    cls_data = formal_model_reduced[input_formal_model['Target'] == cls]
    plt.scatter(cls_data['x1'], cls_data['x2'], s=100, edgecolor='k', color=colors[cls], label=f'Class {cls}')

# Annotate points with ID
for i, txt in enumerate(input_formal_model['ID']):
    plt.annotate(txt, (formal_model_reduced['x1'].iloc[i], formal_model_reduced['x2'].iloc[i]), fontsize=9)

# Draw the separating line
line_x1 = coordinates_df['x1'].tolist()
line_x2 = coordinates_df['x2'].tolist()
plt.plot(line_x1, line_x2, 'ro-')

plt.xlabel("X 1")
plt.ylabel("X 2")
plt.title(f"2D MDS Reduction of {'input_formal_model'}")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


## 4. Calculate Transition Matrix

### 3.1. Singular Value Decomposition (SVD)

In [8]:
# Receive values from the matrix A DataFrame 
formal_model_reduced_values = formal_model_reduced.values

# Calculate SVD
U, s, Vt = np.linalg.svd(formal_model_reduced_values)

# Construct the full diagonal matrix
S = np.zeros(formal_model_reduced_values.shape)
for i in range(min(formal_model_reduced_values.shape)):
    S[i, i] = s[i]

# print("U:\n", U)
# print("\nS:\n", S)
# print("\nVt:\n", Vt)


### 3.2. Create the reconstructed matrix of SVD

In [9]:
# Recompute the original matrix using U, S, and Vt
reconstructed_matrix = U.dot(S.dot(Vt))

# Convert the reconstructed matrix to a pandas DataFrame, if desired
reconstructed_df = pd.DataFrame(reconstructed_matrix)


In [10]:
reconstructed_df

,0,1
0,0.131358,-1.973284
1,0.258753,-2.103057
2,0.388700,-2.232669
3,0.539877,-2.351655
4,0.700758,-2.465481
5,0.994299,1.735109
6,1.249399,1.422603
7,1.640171,1.220070
8,1.963927,1.049257
9,2.385963,0.964396


In [11]:
formal_model_reduced

,x1,x2
0,0.131358,-1.973284
1,0.258753,-2.103057
2,0.388700,-2.232669
3,0.539877,-2.351655
4,0.700758,-2.465481
5,0.994299,1.735109
6,1.249399,1.422603
7,1.640171,1.220070
8,1.963927,1.049257
9,2.385963,0.964396


### 3.3. Calculate the generalized inverse of input matrix

In [12]:
# Calculate the generalized (Moore-Penrose) inverse
formal_model_reduced_pinv = np.linalg.pinv(formal_model_reduced_values)

# Convert to a pandas DataFrame
formal_model_reduced_pinv_df = pd.DataFrame(formal_model_reduced_pinv)

In [13]:
formal_model_reduced_pinv_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-0.002956,0.000110,0.003247,0.007002,0.011041,0.03307,0.039035,0.049090,0.057411,0.068716,-0.046800,-0.050333,-0.053242,-0.056156,-0.059235
1,-0.052279,-0.055322,-0.058353,-0.061029,-0.063534,0.04966,0.042160,0.038049,0.034562,0.033699,0.014732,0.017318,0.019080,0.019744,0.021514


### 3.4. Calculate transition matrix T

In [14]:
transition_matrix_T = formal_model_reduced_pinv_df.dot(formal_model_features)

In [15]:
transition_matrix_T

,f1,f2,f3,f4,f5
0,-0.431169,-0.537241,0.124292,0.310206,1.080180
1,-0.996937,-0.087603,1.259949,-0.367093,-0.265671


## 4. Create Hyperplane

In [16]:
coordinates_hyperplane_df = coordinates_df.values @ transition_matrix_T.values

In [17]:
coordinates_hyperplane_df

array([[ 1.13153726, -0.89927596, -2.27131298,  1.35459821,  2.69170259],
       [ 1.06423723, -0.40583628, -1.76563097,  0.86084542,  1.47868696],
       [ 0.9969372 ,  0.08760341, -1.25994895,  0.36709262,  0.26567133],
       [ 0.92963717,  0.5810431 , -0.75426694, -0.12666017, -0.9473443 ],
       [ 0.86233714,  1.07448278, -0.24858493, -0.62041297, -2.16035994]])

In [18]:
# Adding a new column with all values set to 1
new_column = np.ones((coordinates_hyperplane_df.shape[0], 1))
feature_vector_inverse = np.hstack((new_column, coordinates_hyperplane_df))

feature_vector_inverse

array([[ 1.        ,  1.13153726, -0.89927596, -2.27131298,  1.35459821,
         2.69170259],
       [ 1.        ,  1.06423723, -0.40583628, -1.76563097,  0.86084542,
         1.47868696],
       [ 1.        ,  0.9969372 ,  0.08760341, -1.25994895,  0.36709262,
         0.26567133],
       [ 1.        ,  0.92963717,  0.5810431 , -0.75426694, -0.12666017,
        -0.9473443 ],
       [ 1.        ,  0.86233714,  1.07448278, -0.24858493, -0.62041297,
        -2.16035994]])

In [19]:
# Optimization criteria function
def optimization_criteria(W, X):
    return np.sum(np.abs(X.dot(W.T)))

# Initial guess (starting point for the optimization algorithm)
weights_inverse_initial = np.zeros(feature_vector_inverse.shape[1])

# Bounds for W (setting it between -1 and 1)
bounds = [(-1, 1) for _ in range(feature_vector_inverse.shape[1])]

# The actual optimization
res = minimize(
    optimization_criteria,
    weights_inverse_initial,
    args=(feature_vector_inverse,),
    method='SLSQP',
    bounds=bounds,
    options={'disp': True}
)

# Checking if the optimization was successful
if res.success:
    # Extract the weights into a DataFrame
    weights_inverse_output = pd.DataFrame(res.x, index=[f'w{i}' for i in range(len(res.x))], columns=['weight'])

    print("Optimization was successful. The weights are:")
    print(weights_inverse_output)

    # Output the value of the optimization criteria
    print("\nValue of the optimization criteria (objective function value):", res.fun)
else:
    print("Optimization failed.")

# Checking for convergence
print("\nConvergence status:", res.message)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0717127609123639e-05
            Iterations: 6
            Function evaluations: 72
            Gradient evaluations: 6
Optimization was successful. The weights are:
      weight
w0 -0.000092
w1 -0.000012
w2 -0.000375
w3 -0.000131
w4  0.000096
w5 -0.000246

Value of the optimization criteria (objective function value): 1.0717127609123639e-05

Convergence status: Optimization terminated successfully


In [26]:
weights_inverse_output

,weight
w0,-0.000092
w1,-0.000012
w2,-0.000375
w3,-0.000131
w4,0.000096
w5,-0.000246


In [27]:
# Reorder the weight vector so that w0 is at the end.
weights_df_reordered = pd.concat([weights_inverse_output.iloc[1:], weights_inverse_output.iloc[0:1]]).reset_index(drop=True)

# Show the modified DataFrame
weights_df_reordered


,weight
0,-0.000012
1,-0.000375
2,-0.000131
3,0.000096
4,-0.000246
5,-0.000092


In [28]:
# Create a new DataFrame with an additional column
new_formal_model_features = formal_model_features.copy()
new_formal_model_features['f0'] = 1
new_formal_model_features


,f1,f2,f3,f4,f5,f0
0,2.8,-1.8,-2.800000,1.300000,0.4,1
1,2.9,-1.9,-2.900000,1.400000,0.5,1
2,3.0,-2.0,-3.000000,1.500000,0.6,1
3,3.1,-2.1,-3.100000,1.600000,0.7,1
4,3.2,-2.2,-3.200000,1.700000,0.8,1
5,-1.6,-2.5,1.500000,0.200000,0.6,1
6,-1.3,-2.7,1.300000,0.400000,0.8,1
7,-1.0,-3.0,1.500000,0.600000,1.0,1
8,-0.7,-3.2,1.700000,0.800000,1.2,1
9,-0.5,-3.5,1.900000,1.000000,1.4,1


In [29]:
# # Multiply the features by the weights
my_result = new_formal_model_features.mul(weights_df_reordered['weight'].values, axis=1)

# Sum along the rows to get the final single column for predictions
my_predictions = my_result.sum(axis=1)

# Convert the series to a data frame
my_predictions_df = my_predictions.to_frame(name='Prediction')

In [30]:
my_predictions_df = pd.concat([my_predictions_df, formal_model_target], axis=1)


In [31]:
my_predictions_df

,prediction,Target
0,0.000942,1
1,0.000976,1
2,0.001010,1
3,0.001045,1
4,0.001079,1
5,0.000539,2
6,0.000606,2
7,0.000659,2
8,0.000674,2
9,0.000728,2
